In [1]:
# 경로설정
import sys
path = "C:\github/aide-analysis/"
path2 = 'C:\github/aide-analysis/notebook/temp_ljs/Develop/지주리포트Lite/실거래 이상값 제거 연구 및 추론/모듈 및 압축파일 보관 폴더'
sys.path.append(path)
sys.path.append(path2)

import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd())))
# 한글폰트
import os
import matplotlib.pyplot as plt
if os.name =='posix':
    plt.rc('font',family = 'ApplGothic')
else:
    plt.rc('font',family = 'Malgun Gothic')
    
# 경고출력문 제거
import warnings
warnings.filterwarnings("ignore")

# 모듈 import
import pandas as pd
import src
from src.openapi import KakaoLocalAPI as kakao
from openpyxl import load_workbook
from src.utils import insert_dataframe_to_excel_sheet
from src.sql import DatabaseHandler
from src.temp import *
from datetime import datetime
import datetime
from dateutil.relativedelta import relativedelta
from src.bigquery import BigqueryHandler
import seaborn as sns
import numpy as np
import statistics
from tqdm import tqdm
import config
from core.config import Config
import json
from datetime import datetime
import geopandas as gpd
from tqdm import tqdm
import folium
from src.bigquery import bigquery_client, bigquery
import geopandas as gpd
from shapely import wkt
from openpyxl import load_workbook
from src.utils import insert_dataframe_to_excel_sheet
from openpyxl.utils.dataframe import dataframe_to_rows
from difflib import SequenceMatcher
import openpyxl
import math
from 실거래전처리 import TradeProcessing

config = Config()
BH = BigqueryHandler()


# DataFrame 디스플레이 설정
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)

### 최근 3달 가장 거래가 활발한 단지

In [73]:
sql =f"""
SELECT T1.*
FROM(
    SELECT complexNo, 
           complexName, 
           areaSixGroupNo, 
           useApproveYear, 
           totalHouseholdCount, 
           tradeType, 
           contractDate, 
           averagePyeong, 
           averagePrice, 
           isFloor, 
           isProcessing1, 
           isProcessing2, 
           isProcessing3, 
           interpolate, 
           yearMonth, 
           isReal, 
           finalProcessing, 
           predict,
           RANK() OVER(PARTITION BY complexNo, complexName, areaSixGroupNo ORDER BY yearMonth DESC) AS rn -- 월 구분
    FROM `aidepartners.raw.aide_apartment_price_origin`
    WHERE sidoCode ='11' AND yearMonth >='2023-01-01' AND isReal = True
    ORDER BY complexNo, areaSixGRoupNo
    ) AS T1
"""
df = BH.read_table(sql)

In [74]:
df_result = pd.DataFrame()
df_unique  = df[['complexNo','areaSixGroupNo']].drop_duplicates().reset_index(drop =True)
for i in range(0, len(df_unique)):
    cNo =df_unique['complexNo'].iloc[i]
    area = df_unique['areaSixGroupNo'].iloc[i]
    # 거래가 두달이상 연속으로 있는경우
    if len(df.loc[(df['complexNo']==cNo)&(df['areaSixGroupNo']==area), 'rn'].unique()) >1:
        # 최근거래과 과거거래 금액 차이 비교
        recentPrice = df.loc[(df['complexNo']==cNo)&(df['areaSixGroupNo']==area)&(df['rn']==1),'averagePyeong'].mean()
        pastPrice = df.loc[(df['complexNo']==cNo)&(df['areaSixGroupNo']==area)&(df['rn']==2),'averagePyeong'].mean()
        price = pastPrice - recentPrice
        if price >0 :
            df_result = df_result.append(df[(df['complexNo']==cNo)&(df['areaSixGroupNo']==area)])

In [77]:
df_result = df_result[['complexNo','complexName','areaSixGroupNo']].drop_duplicates()

In [78]:
df_result

,complexNo,complexName,areaSixGroupNo
132,107929,롯데캐슬골드파크1차,A3
269,113907,고덕그라시움,A3
333,117792,신길센트럴자이,A2
335,117842,고덕센트럴푸르지오(주상복합),A2
387,119588,사가정센트럴아이파크,A3
402,121977,고덕자이,A3
418,1250,동성7차,A2
640,26159,길음뉴타운8단지래미안,A2
655,26638,길음서희스타힐스(주상복합),A3
675,27212,둔촌푸르지오,A3


이렇게 면적을 그룹으로 나눈거에서 평단가로 했으며 월별 평균대표값으로 했을때 문제가 통계적 오류가 생길 게 있을까?